In [1]:
import requests
import re
import pymorphy2
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from fake_useragent import UserAgent
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
import gensim.downloader as api
from gensim.models import Word2Vec

In [2]:
ua = UserAgent(verify_ssl=False)
headers = {'User-Agent': ua.random}

### 1. Сбор данных

Я выбрала сайт https://www.turpravda.com/tn/top-hotels.html с отзывами на отели Туниса. Оттуда я взяла страницы с отелями, которые оценивались на 9-10, 7, 6 и 1-5 баллов в среднем. На каждой странице максимум 25 отелей. 

Так как страницы с самими отзывами позволяют просматривать новые отзывы только по нажатию специальной кнопки, пришлось брать только по 10 первых отзывов с каждой страницы, так как я не нашла функционала, с помощью которого можно достать больше отзывов.

In [3]:
def get_comments(page_url):
    global df_comments
    
    session = requests.session()
    req = session.get(page_url, headers=headers)
    page = req.text
    soup = BeautifulSoup(page)
    
    for comment in soup.find_all('div', {'class': 'ans_body'}):
        mark = comment.find('span', {'class': 'value'})
        if mark:
            comment_text = comment.find('span', {'class': 'all-text'}).text
            mark = float(mark.text[-4:])
            df_comments = df_comments.append({'comment': comment_text, 
                                              'mark': mark,
                                              'url': page_url}, ignore_index=True)

In [4]:
df_comments = pd.DataFrame(columns=['comment', 'mark', 'url'])
rates = [9, 7, 6, 5, '5&p=2']

for i in rates:
    page_url = f'https://www.turpravda.com/tn/top-hotels.html?rte%5B%5D={i}'
    session = requests.session()
    req = session.get(page_url, headers=headers)
    page = req.text
    soup = BeautifulSoup(page)

    all_hrefs = soup.find_all('a', {'class': 'hotel-name-title'})
    for href in tqdm(all_hrefs):
        link = href.get('href')
        get_comments('https://www.turpravda.com' + link)

100%|██████████| 25/25 [00:50<00:00,  2.02s/it]


Таким образом, у меня получилось почти равное количество положительных и отрицательных отзывов. К отрицательным я относила те отзывы, в которых оценка от 1 до 5, включительно. Остальные я относила к положительным.

In [5]:
df_comments['sentiment'] = df_comments['mark'].apply(lambda x: 1 if x > 5 else 0)
df_comments['sentiment'].value_counts()

1    487
0    422
Name: sentiment, dtype: int64

### 2. Создание словаря

In [6]:
m = pymorphy2.MorphAnalyzer()
sw = stopwords.words('russian')
def clean_text(text):
    tokens = []
    for word in word_tokenize(text):
        if word.isalpha():
            if word not in sw:
                tokens.append(m.parse(word.lower())[0].normal_form)
    return tokens

In [7]:
df_comments['tokens'] = df_comments['comment'].apply(clean_text)
df_comments['clean_comment'] = df_comments['tokens'].apply(lambda x: ' '.join(x))

In [8]:
X = df_comments['tokens']
y = df_comments['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state=8)

In [9]:
mask_pos = y==1
positive_comments = X_train[mask_pos].tolist()
negative_comments = X_train[~mask_pos].tolist()

positive_corpus = []
negative_corpus = []

for t in positive_comments:
    positive_corpus.extend(t)
for t in negative_comments:
    negative_corpus.extend(t)

In [29]:
cnt_pos = Counter(positive_corpus).most_common(300)
cnt_neg = Counter(negative_corpus).most_common(300)

set_pos = set(dict(cnt_pos).keys())
set_neg = set(dict(cnt_neg).keys())
print('Only positive:')
print(set_pos-set_neg)
print('Only negative:')
print(set_neg-set_pos)

Only positive:
{'турция', 'неплохой', 'белый', 'сахар', 'быстро', 'ну', 'кухня', 'спасибо', 'отличный', 'внимание', 'интересный', 'английский', 'расположить', 'такси', 'выбрать', 'для', 'зелёный', 'именно', 'танец', 'центр', 'приятный', 'недостаток', 'хаммамет', 'прекрасный', 'разнообразный', 'сладость', 'шоу', 'уровень', 'уходить', 'бесплатно', 'французский', 'народ', 'замечательный', 'супер', 'горка', 'бесплатный', 'ребята', 'вкусно', 'вход', 'линия', 'приветливый', 'всякий', 'обязательно', 'вернуться', 'медина', 'некоторый', 'целое', 'спокойный', 'берег', 'тёплый', 'покупать', 'метр', 'довольный', 'до', 'вполне', 'чисто', 'качество', 'небольшой', 'язык', 'француз', 'дешёвый', 'любой', 'поездка', 'порт', 'достаточно', 'отлично', 'голодный', 'погода', 'новый', 'обслуживание', 'кормить', 'пройти'}
Only negative:
{'нечего', 'полный', 'шезлонг', 'тип', 'ванная', 'найти', 'остальной', 'музыка', 'принцип', 'приходить', 'грязь', 'друг', 'к', 'стоять', 'рука', 'прийтись', 'увидеть', 'следующ

In [43]:
only_pos = set_pos - set_neg
only_neg = set_neg - set_pos
def sentiment(comments):
    cnt_pos = 0
    cnt_neg = 0
    result = []
    for i, comment in enumerate(comments):
        for token in comment:
            if token in only_pos:
                cnt_pos += 1
            elif token in only_neg:
                cnt_neg += 1
        if cnt_neg > cnt_pos:
            result.append(0)
        else:
            result.append(1)
        cnt_pos = 0
        cnt_neg = 0
    return result

y_pred = sentiment(X_test)

In [44]:
def accuracy_count(y_pred, y_test):
    cnt = 0
    for p, t in zip(y_pred, y_test):
        if p == t:
            cnt += 1
    return cnt / len(y_pred)

accuracy_count(y_pred, y_test)

0.7472527472527473

Полученный способ позволил достичь accuracy 0.747. 

### Добавление моделей

С помощью словарей мы обращаем внимание на количество слов, принадлежащих одному или другому классу, однако, возможно важно смотреть не столько на количество всех слов одного класса, сколько на количество определённых слов по отдельности. Поэтому я предлагаю закодировать вектора с помощью CountVectorizer, чтобы обращать внимание на все слова в комментарии, и предсказывать с помощью Logistic Regression, который после обучения сможет поставить коэффициенты на те слова, которые наиболее сильно влияют на результат.

In [13]:
X_comments = df_comments['clean_comment']
y_comments = df_comments['sentiment']
X_comments_train, X_comments_test, y_comments_train, y_comments_test = train_test_split(X_comments, y_comments, train_size=.8, random_state=1)

In [14]:
vec = CountVectorizer()
X_train_vec = vec.fit_transform(X_comments_train)
X_test_vec = vec.transform(X_comments_test)

clf = LogisticRegression(solver='newton-cg')
clf.fit(X_train_vec, y_comments_train)
y_preds = clf.predict(X_test_vec)

In [15]:
accuracy_count(y_preds, y_comments_test)

0.8571428571428571

Качество улучшилось на 10%

Аналогичный результат получается при использовании баесовской модели MultinominalNB

In [16]:
clf = MultinomialNB()
clf.fit(X_train_vec, y_comments_train)
y_preds = clf.predict(X_test_vec)
accuracy_count(y_preds, y_comments_test)

0.8626373626373627

Также я решила попробовать обучить модель определять тональность комментария на основе эмбеддинга самого комментария. Для этого я взяла обученную модель Word2Vec, подстроила данные под входные данные для модели и каждый отзыв представила как средний вектор всех представлений слов. После я обучила модель Logistic Regression предсказывать тональность.

In [54]:
model = api.load('word2vec-ruscorpora-300')

# Перевод pos-тэгов pymorphy в pos-тэги модели (причастия и деепричастия я считала глаголами)
pos_model = {'NOUN': 'NOUN', 'VERB': 'VERB', 'ADJF': 'ADJ', 
            'ADJS': 'ADJ', 'PRTS': 'VERB', 'COMP': 'ADJ',
            'ADVB': 'ADV', 'INFN': 'VERB', 'NPRO': 'PRON', 
            'PREP':'ADP', 'PRED': 'ADV', 'PRTF': 'VERB',
            'NUMR': 'NUM', 'CONJ': 'CCONJ', 'INTJ': 'INTJ', 
            'PRCL': 'PART', 'GRND': 'VERB'}

# создание датасета для модели
# к каждому слову в конце добавляется тэг части речи
X_train_wv = []
for comment in tqdm(X_train):
    new_comment = []
    for word in comment:
        pos = m.parse(word)[0].tag.POS
        if pos:
            new_comment.append(f'{word}_{pos_model[pos]}')
    X_train_wv.append(new_comment)

X_test_wv = []
for comment in tqdm(X_test):
    new_comment = []
    for word in comment:
        pos = m.parse(word)[0].tag.POS
        if pos:
            new_comment.append(f'{word}_{pos_model[pos]}')
    X_test_wv.append(new_comment)

100%|██████████| 182/182 [00:09<00:00, 18.48it/s]


In [51]:
empts = []  # список слов, которых нет в словаре модели (с повторениями)
all_words = 0  # счётчик общего количества слов (с повторениями)

# создание вектора отзыва
# случаи, когда ни одно слово не входило в словарь, исключались
X_train_emb = []
y_train_emb = []
for comment, y in zip(X_train_wv, y_train):
    comment_emb = []
    all_words += len(comment)
    for word in comment:
        try:
            comment_emb.append(model[word])
        except:
            empts.append(word)
    if comment_emb != []:
        X_train_emb.append(np.mean(comment_emb, axis=0))
        y_train_emb.append(y)

X_test_emb = []
y_test_emb = []
for comment, y in zip(X_test_wv, y_test):
    comment_emb = []
    all_words += len(comment)
    for word in comment:
        try:
            comment_emb.append(model[word])
        except:
            empts.append(word)
    if comment_emb != []:
        X_test_emb.append(np.mean(comment_emb, axis=0))
        y_test_emb.append(y)

In [52]:
clf = LogisticRegression(solver='newton-cg')
clf.fit(X_train_emb, y_train_emb)
y_preds = clf.predict(X_test_emb)
accuracy_count(y_preds, y_test_emb)

0.7833333333333333

Таким образом, accuracy улучшилась на 4 процента по сравнению с изначальным предсказанием.

In [53]:
print('Слова не из словаря:', len(empts))
print('Слова в словаре:', all_words)

Слова не из словаря: 42558
Слова в словаре: 189202
